In [95]:
import chromadb

client = chromadb.CloudClient(
  api_key='ck-HqbGPjsNv7eWuDFf7NjqcFuV4pbdDSk51wtwo5hX6Y9B',
  tenant='ee54a044-1346-4b7c-89b7-d7554fb9aacf',
  database='RAG-chiangmai-travel'
)

# Load Several Json file from Folder

In [ ]:
import os
import json
import jq
from dotenv import load_dotenv
from langchain_community.document_loaders import JSONLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer


## Embedding

In [123]:
from pprint import pprint
load_dotenv()

True

In [152]:
path_to_json = "/Users/ploy/Programming/thesis/data/json-scrape/hotel"  # Directory containing your JSON files
collection_name = "hotel"

# Get list of JSON files
json_files = [f for f in os.listdir(path_to_json) if f.endswith('.json')]
print(f"Found {len(json_files)} JSON files: {json_files}")

# Cell 5: Initialize embedding model
# Using a Thai-compatible embedding model
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'},  # Use 'cuda' if you have GPU
    encode_kwargs={'normalize_embeddings': True}
)

Found 36 JSON files: ['Chiang_Dao_Good_View_Resort.json', 'Flora_Creek.json', 'Onsen_at_Moncham.json', 'Meliá_Chiang_Mai.json', 'The_Inside_House.json', 'โรงแรม_ลานนา_ดุสิตา_ริเวอร์ไซด์_บูทิค_รีสอร์ท_(Lanna_Dusita_Riverside_Boutique_Resort).json', 'ยู_นิมมาน_เชียงใหม่_(U_Nimman_Chiang_Mai).json', 'วินทรี_ซิตี้_รีสอร์ท_เชียงใหม่_(Wintree_City_Resort_Chiangmai).json', 'เก๊าไม้ล้านนารีสอร์ท.json', 'Hop_In_Chiang_Mai_Airport.json', 'โรงแรมศาลาล้านนา_(Sala_Lanna_Hotel).json', 'ปานวิมานเชียงใหม่_(Panviman_Chiangmai_Spa_Resort).json', 'Buri_Siri_Boutique_Hotel.json', 'ธรรมดาโฮมสเตย์.json', 'โรงแรมบุรีศรีปิง_ริเวอร์ไซด์_รีสอร์ท_แอนด์_สปา.json', 'บ้านระเบียงจำปา.json', 'โรงแรมดาราเทวีเชียงใหม่_(Dara_Thewi_Chiang_Mai_Hotel).json', 'Budecha_(บูเดอช่า_บ้านใกล้ชิดดวงดาว).json', 'ฮิมดอยโฮม.json', 'ม่อนเหนือโฮมสเตย์_(Monnuea_Homestay).json', 'Intercontinental_Chiang_Mai_Mae_Ping_Hotel.json', 'ม่อนอิงดาวรีสอร์ท_(Mon_Ing_Dao_Resort).json', 'บ้านระเบียงดาว.json', 'โรงแรมเลอเมอริเดียน_เชียงใหม่_(Le_Merid

In [141]:
def process_travel_json(file_path):
    """
    Custom function to process travel data JSON files
    """
    documents = []

    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Handle both single object and array of objects
    if isinstance(data, dict):
        data = [data]

    for item in data:
        # Create text content by combining all fields
        text_parts = []

        # Add name
        if 'name' in item and item['name']:
            text_parts.append(f"ชื่อ: {item['name']}")

        # Add address
        if 'address' in item and item['address']:
            text_parts.append(f"ที่อยู่: {item['address']}")

        # Add opening hours
        if 'store_opening_and_closing' in item and item['store_opening_and_closing']:
            text_parts.append(f"เวลาเปิด-ปิด: {item['store_opening_and_closing']}")

        # Add rating
        if 'Review_rating' in item and item['Review_rating']:
            text_parts.append(f"คะแนนรีวิว: {item['Review_rating']}")

        # Add comment/review
        if 'Comment_review' in item and item['Comment_review']:
            text_parts.append(f"รีวิว: {item['Comment_review']}")

        # Combine all text
        page_content = "\n".join(text_parts)

        # Create metadata
        metadata = {
            'source': file_path,
            'name': item.get('name', ''),
            'address': item.get('address', ''),
            'rating': item.get('Review_rating', ''),
            'opening_hours': item.get('store_opening_and_closing', '')
        }

        # Create document object
        from langchain.schema import Document
        doc = Document(page_content=page_content, metadata=metadata)
        documents.append(doc)

    return documents

In [142]:
all_documents = []

print("Processing JSON files...")
for file in json_files:
    file_path = os.path.join(path_to_json, file)
    print(f"Processing: {file}")

    try:
        documents = process_travel_json(file_path)
        all_documents.extend(documents)
        print(f"  - Loaded {len(documents)} documents from {file}")
    except Exception as e:
        print(f"  - Error processing {file}: {str(e)}")

print(f"\nTotal loaded: {len(all_documents)} documents from JSON files.")


Processing JSON files...
Processing: สวนสนุก_FA_POP_By_FUNplanet_(สวนสนุก_ฟันแพลนเน็ต).json
  - Loaded 1 documents from สวนสนุก_FA_POP_By_FUNplanet_(สวนสนุก_ฟันแพลนเน็ต).json
Processing: วัดศรีประดิษฐ์_(วัดศรีประดิษฐ์_ป่าคา).json
  - Loaded 1 documents from วัดศรีประดิษฐ์_(วัดศรีประดิษฐ์_ป่าคา).json
Processing: น้ำตกศรีสังวาลย์.json
  - Loaded 1 documents from น้ำตกศรีสังวาลย์.json
Processing: เพลินฤดี_ไนท์_มาร์เก็ต_(เพลินฤดี).json
  - Loaded 1 documents from เพลินฤดี_ไนท์_มาร์เก็ต_(เพลินฤดี).json
Processing: สำนักปฏิบัติธรรมดอยเทพเนรมิต.json
  - Loaded 1 documents from สำนักปฏิบัติธรรมดอยเทพเนรมิต.json
Processing: สถานีขนส่งเชียงดาว.json
  - Loaded 1 documents from สถานีขนส่งเชียงดาว.json
Processing: ศูนย์วิจัยข้าวสะเมิง.json
  - Loaded 1 documents from ศูนย์วิจัยข้าวสะเมิง.json
Processing: วัดดอยอุดมธรรม.json
  - Loaded 1 documents from วัดดอยอุดมธรรม.json
Processing: ศูนย์พัฒนาโครงการหลวง_ขุนแปะ.json
  - Loaded 1 documents from ศูนย์พัฒนาโครงการหลวง_ขุนแปะ.json
Processing: สวนสัตว์แ

In [143]:
if all_documents:
    print("Sample document:")
    print(f"Content: {all_documents[0].page_content[:200]}...")
    print(f"Metadata: {all_documents[0].metadata}")

Sample document:
Content: ชื่อ: สวนสนุก FA POP By FUNplanet (สวนสนุก ฟันแพลนเน็ต)
ที่อยู่: 2, ถนนมหิดล, ตำบลป่าแดด, เชียงใหม่, จ.เชียงใหม่, 50100, ประเทศไทย เชียงใหม่
เวลาเปิด-ปิด: ปิดอยู่
คะแนนรีวิว: 5.0
รีวิว: สวนสนุก FA POP...
Metadata: {'source': '/Users/ploy/Programming/thesis/data/json-scrape/attraction/สวนสนุก_FA_POP_By_FUNplanet_(สวนสนุก_ฟันแพลนเน็ต).json', 'name': 'สวนสนุก FA POP By FUNplanet (สวนสนุก ฟันแพลนเน็ต)', 'address': '2, ถนนมหิดล, ตำบลป่าแดด, เชียงใหม่, จ.เชียงใหม่, 50100, ประเทศไทย เชียงใหม่', 'rating': '5.0', 'opening_hours': 'ปิดอยู่'}


In [144]:
try:
    # Initialize ChromaDB client for cloud
    client = chromadb.CloudClient(
  api_key='ck-HqbGPjsNv7eWuDFf7NjqcFuV4pbdDSk51wtwo5hX6Y9B',
  tenant='ee54a044-1346-4b7c-89b7-d7554fb9aacf',
  database='RAG-chiangmai-travel'
)

    print("Connected to ChromaDB Cloud successfully!")
    print(f"Server version: {client.get_version()}")

except Exception as e:
    print(f"Failed to connect to ChromaDB Cloud: {str(e)}")
    print("Please check your credentials and connection settings.")

Connected to ChromaDB Cloud successfully!
Server version: 1.0.0


In [145]:
try:
    # Try to get existing collection
    try:
        collection = client.get_collection(name=collection_name)
        print(f"Found existing collection: {collection_name}")
        print(f"Collection count: {collection.count()}")
    except:
        # Create new collection if it doesn't exist
        collection = client.create_collection(
            name=collection_name,
            metadata={"description": "Chiang Mai travel places and reviews"}
        )
        print(f"Created new collection: {collection_name}")

except Exception as e:
    print(f"Error with collection: {str(e)}")

Found existing collection: attraction
Collection count: 86


In [146]:
vectorstore = Chroma(
    client=client,
    collection_name=collection_name,
    embedding_function=embedding_model,
)

print("Vectorstore initialized!")

Vectorstore initialized!


In [150]:
batch_size = 10  # Adjust based on your needs and API limits

print(f"Adding {len(all_documents)} documents to ChromaDB in batches of {batch_size}...")

for i in range(0, len(all_documents), batch_size):
    batch = all_documents[i:i + batch_size]

    try:
        # Add documents to vectorstore
        vectorstore.add_documents(batch)
        print(f"Added batch {i//batch_size + 1}: documents {i+1} to {min(i+batch_size, len(all_documents))}")

    except Exception as e:
        print(f"Error adding batch {i//batch_size + 1}: {str(e)}")
        # Continue with next batch

print("All documents have been processed!")

Adding 99 documents to ChromaDB in batches of 10...
Added batch 1: documents 1 to 10
Added batch 2: documents 11 to 20
Added batch 3: documents 21 to 30
Added batch 4: documents 31 to 40
Added batch 5: documents 41 to 50
Added batch 6: documents 51 to 60
Added batch 7: documents 61 to 70
Added batch 8: documents 71 to 80
Added batch 9: documents 81 to 90
Added batch 10: documents 91 to 99
All documents have been processed!


In [151]:
try:
    collection_info = client.get_collection(collection_name)
    total_count = collection_info.count()
    print(f"\nVerification:")
    print(f"Total documents in collection '{collection_name}': {total_count}")

    # Test similarity search
    if total_count > 0:
        test_query = "ร้านอาหารในเชียงใหม่"
        results = vectorstore.similarity_search(test_query, k=3)

        print(f"\nTest search for '{test_query}':")
        for i, doc in enumerate(results, 1):
            print(f"{i}. {doc.metadata.get('name', 'Unknown')}")
            print(f"   Content: {doc.page_content[:100]}...")
            print(f"   Rating: {doc.metadata.get('rating', 'N/A')}")
            print()

except Exception as e:
    print(f"Error during verification: {str(e)}")


Verification:
Total documents in collection 'attraction': 284

Test search for 'ร้านอาหารในเชียงใหม่':
1. เพลินฤดี ไนท์ มาร์เก็ต (เพลินฤดี)
   Content: ชื่อ: เพลินฤดี ไนท์ มาร์เก็ต (เพลินฤดี)
ที่อยู่: 45 ถนนอัษฎาธร ต.ป่าตัน เชียงใหม่ (ตรงข้ามจริงใจมาร์...
   Rating: 3.6

2. เพลินฤดี ไนท์ มาร์เก็ต (เพลินฤดี)
   Content: ชื่อ: เพลินฤดี ไนท์ มาร์เก็ต (เพลินฤดี)
ที่อยู่: 45 ถนนอัษฎาธร ต.ป่าตัน เชียงใหม่ (ตรงข้ามจริงใจมาร์...
   Rating: 3.6

3. ตลาดต้นลำไย
   Content: ชื่อ: ตลาดต้นลำไย
ที่อยู่: 90 ถนน วิชยานนท์ เชียงใหม่ (กาดต้นลำไยตั้งอยู่ตรงข้ามกาดหลวงทางฝั่งถนนวิช...
   Rating: 3.5



# Test Retrieval 

In [134]:
def search_similar_places(query, k=5):
    """
    Search for similar places based on query
    """
    try:
        results = vectorstore.similarity_search(query, k=k)

        print(f"Search results for: '{query}'")
        print("-" * 50)

        for i, doc in enumerate(results, 1):
            print(f"{i}. {doc.metadata.get('name', 'Unknown')}")
            print(f"   Address: {doc.metadata.get('address', 'N/A')}")
            print(f"   Rating: {doc.metadata.get('rating', 'N/A')}")
            print(f"   Hours: {doc.metadata.get('opening_hours', 'N/A')}")
            print(f"   Content: {doc.page_content[:150]}...")
            print()

        return results

    except Exception as e:
        print(f"Error during search: {str(e)}")
        return []

In [135]:
def search_with_score(query, k=5):
    """
    Search with similarity scores
    """
    try:
        results = vectorstore.similarity_search_with_score(query, k=k)

        print(f"Search results with scores for: '{query}'")
        print("-" * 50)

        for i, (doc, score) in enumerate(results, 1):
            print(f"{i}. {doc.metadata.get('name', 'Unknown')} (Score: {score:.4f})")
            print(f"   Content: {doc.page_content[:100]}...")
            print()

        return results

    except Exception as e:
        print(f"Error during scored search: {str(e)}")
        return []

In [136]:
search_similar_places("ร้านอาหารเชียงใหม่", k=3)

Search results for: 'ร้านอาหารเชียงใหม่'
--------------------------------------------------
1. เขยเจียงใหม่ อาหารเหนือ (ร้านอาหารเชียงใหม่)สาขาสันติธรรม
   Address: 14 ถ.สันติธรรม ต.ช้างเผือก อ.เมือง จ.เชียงใหม่ 50300 เชียงใหม่ (อยู่ใกล้เมาทะเล ติดกับ Sunshine House)
   Rating: 4
   Hours: 11:00 - 22:00
   Content: ชื่อ: เขยเจียงใหม่ อาหารเหนือ (ร้านอาหารเชียงใหม่)สาขาสันติธรรม
ที่อยู่: 14 ถ.สันติธรรม ต.ช้างเผือก อ.เมือง จ.เชียงใหม่ 50300 เชียงใหม่ (อยู่ใกล้เมาทะ...

2. ภัตตาคารเจี่ยท้งเฮง (Jia Tong Heng Restaurant)ศรีดอนไชย
   Address: 193/2-3 ถ.ศรีดอนไชย เชียงใหม่ (ร้านอยู่ริมถนนเยื้องกับทางออกโรงแรมเชียงใหม่พลาซ่า ข้างวัดศรีดอนไชย)
   Rating: 4.2
   Hours: ปิดอยู่จะเปิดในเวลา 10:30
   Content: ชื่อ: ภัตตาคารเจี่ยท้งเฮง (Jia Tong Heng Restaurant)ศรีดอนไชย
ที่อยู่: 193/2-3 ถ.ศรีดอนไชย เชียงใหม่ (ร้านอยู่ริมถนนเยื้องกับทางออกโรงแรมเชียงใหม่พลาซ...

3. ก๋วยเตี่ยวเก่ง-กิ๊ฟ บ้านน้อยซอยลึกสาขา1
   Address: 73\2หมู่3ต.แม่เหียะอ.เมืองจ.เชียงใหม่50100 เชียงใหม่
   Rating: 4.0
   Hours: ทุกวัน

[Document(id='82fd8fce-c6c5-49e4-adc1-bc55e39e0ef7', metadata={'rating': '4', 'name': 'เขยเจียงใหม่ อาหารเหนือ (ร้านอาหารเชียงใหม่)สาขาสันติธรรม', 'source': '/Users/ploy/Programming/thesis/data/json-scrape/resturant/เขยเจียงใหม่_อาหารเหนือ_(ร้านอาหารเชียงใหม่)สาขาสันติธรรม.json', 'address': '14 ถ.สันติธรรม ต.ช้างเผือก อ.เมือง จ.เชียงใหม่ 50300 เชียงใหม่ (อยู่ใกล้เมาทะเล ติดกับ Sunshine House)', 'opening_hours': '11:00 - 22:00'}, page_content='ชื่อ: เขยเจียงใหม่ อาหารเหนือ (ร้านอาหารเชียงใหม่)สาขาสันติธรรม\nที่อยู่: 14 ถ.สันติธรรม ต.ช้างเผือก อ.เมือง จ.เชียงใหม่ 50300 เชียงใหม่ (อยู่ใกล้เมาทะเล ติดกับ Sunshine House)\nเวลาเปิด-ปิด: 11:00 - 22:00\nคะแนนรีวิว: 4\nรีวิว: นักท่องเที่ยงทั้งคนไทยคนต่างชาติที่เชียงใหม่น้อยลงอย่างน่าใจหาย เป็นมาได้สักพัก ไม่ใช่เฉพาะช่วงนี้ และที่ร้านเขยเจียงใหม่ ในย่านสันติธรรม ก็เช่นกัน\n\nเขยเจียงใหม่ เป็นร้านอาหารเหนือที่เราแอบตั้งคำถาม คิดว่าทางร้านเข้าใจสรรหาเมนูที่ชื่อค่อนข้างพิศดารชวนว้าว ให้คนคิดอยากลอง แต่สำหรับรสชาติเรามองว่าไม่ได้เป็นไปทางใดทางหนึ่ง 

In [137]:
search_with_score("ร้านอาหารเหนือ", k=5)

Search results with scores for: 'ร้านอาหารเหนือ'
--------------------------------------------------
1. Mix Beef Club & Restaurant (มิกซ์บีฟคลับแอนด์เรสเตอร์รอง)นิมมานเหมินท์ ซอย 1 (Score: 0.7810)
   Content: ชื่อ: Mix Beef Club & Restaurant (มิกซ์บีฟคลับแอนด์เรสเตอร์รอง)นิมมานเหมินท์ ซอย 1
ที่อยู่: 36 ถนน น...

2. Baan Landai Fine Thai Cuisine (บ้านลันได) (Score: 0.7837)
   Content: ชื่อ: Baan Landai Fine Thai Cuisine (บ้านลันได)
ที่อยู่: 252/13-14 Prapokklao Rd ถนน ศรีภูมิ อำเภอเม...

3. เขยเจียงใหม่ อาหารเหนือ (ร้านอาหารเชียงใหม่)สาขาสันติธรรม (Score: 0.8165)
   Content: ชื่อ: เขยเจียงใหม่ อาหารเหนือ (ร้านอาหารเชียงใหม่)สาขาสันติธรรม
ที่อยู่: 14 ถ.สันติธรรม ต.ช้างเผือก ...

4. ร้าน เนื้อกระทะมุสลิม (Muslim Beef Grill) (Score: 0.8374)
   Content: ชื่อ: ร้าน เนื้อกระทะมุสลิม (Muslim Beef Grill)
ที่อยู่: 380, Thanon Charoen Muang, เชียงใหม่ (ที่เด...

5. Makad Rooftop Cafe มากาด ดาดฟ้า คาดฟ่ - ช้างม่อย (Score: 0.8799)
   Content: ชื่อ: Makad Rooftop Cafe มากาด ดาดฟ้า คาดฟ่ - ช้างม่อย
ที

[(Document(id='80bbcd90-0aed-41f6-9025-cc5f61d6a459', metadata={'opening_hours': 'ปิดอยู่จะเปิดในเวลา 11:00', 'source': '/Users/ploy/Programming/thesis/data/json-scrape/resturant/Mix_Beef_Club_&_Restaurant_(มิกซ์บีฟคลับแอนด์เรสเตอร์รอง)นิมมานเหมินท์_ซอย_1.json', 'address': '36 ถนน นิมมานเหมินท์ ซอย 1 เชียงใหม่ (สุดซอยนิมมานเหมินทร์ซอย 1)', 'rating': '4.2', 'name': 'Mix Beef Club & Restaurant (มิกซ์บีฟคลับแอนด์เรสเตอร์รอง)นิมมานเหมินท์ ซอย 1'}, page_content='ชื่อ: Mix Beef Club & Restaurant (มิกซ์บีฟคลับแอนด์เรสเตอร์รอง)นิมมานเหมินท์ ซอย 1\nที่อยู่: 36 ถนน นิมมานเหมินท์ ซอย 1 เชียงใหม่ (สุดซอยนิมมานเหมินทร์ซอย 1)\nเวลาเปิด-ปิด: ปิดอยู่จะเปิดในเวลา 11:00\nคะแนนรีวิว: 4.2\nรีวิว: ร้านตั้งอยู่\u200bด้านในสุดของนิมมานเหมินท์\u200b ซอย 1 มีชื่อเรื่องเมนู\u200bเนื้อวัวและไวน์\u200b จำหน่าย\u200bเมนู\u200bอาหาร\u200bไทย ฝรั่ง และ fusion มีที่นั่งทั้ง\u200bแบบ\u200b และ\u200b จำนวนมากพอสมควร\u200b เมนู\u200bเนื้อที่สั่งนุ่มใช้ได้ ไวน์มีให้เลือกหลากหลาย\u200bและดีสมคำร่ำลือ ขนมเป็น\u200bอีกรายกา